In [ ]:
import os

dir_root = './data'
image_root = dir_root + "/train/images"
label_root = dir_root + "/train/labels"

# 클래스 이름과 클래스 인덱스 매핑
product_list = os.listdir(image_root)
for i in range(len(product_list)):
    product_list[i] = product_list[i][6:]

In [4]:
import yaml

data = {
    'train': 'C:/capstone/data/processed/images/train',
    'val': 'C:/capstone/data/processed/images/validation',
    'nc': len(product_list),
    'names': product_list
}

with open('data.yaml', 'w', encoding='utf-8') as f:
    yaml.dump(data, f, allow_unicode=True)

In [ ]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())

In [ ]:
from ultralytics import YOLO

# Load a YOLO model
saved = 'runs/detect/product_detector5/weights/best.pt'
model = YOLO(saved)

results = model.train(data="data.yaml", epochs=10, batch=8, imgsz=640, device=0, name='product_detector')

# Evaluate the model's performance on the validation set
results = model.val()

# Export the model to ONNX format
# success = model.export(format="onnx", device=0)

In [ ]:
### Legacy
 
'''
data_yaml = """
train: C:/capstone/data/processed/images/train
val: C:/capstone/data/processed/images/validation

"""

data_yaml += 'nc: ' + str(len(product_list)) + '\n'
data_yaml += 'names: ' + str(product_list) + '\n'

with open("data.yaml", "w") as f:
    f.write(data_yaml)

# n: 72
# s: 74
# m: 65

saved = 'product-detector/runs/train/product_detector65/weights/best.pt'
!python product-detector/train.py --img 640 --batch 8 --epochs 1 --data data.yaml --weights yolo11n.pt --name product_detector --device 0
# !cp runs/train/product_detector/weights/best.pt {dir_root}

from pathlib import Path
import torch

# best 모델 경로
best_model_path = 'C:/capstone/product-detector/runs/train/product_detector15/weights/best.pt'

# 모델 로드
# model = torch.load(best_model_path, map_location='cpu', weights_only=False)
# model = torch.hub.load('ultralytics\\yolov5', 'custom', path='runs/train/product_detector16/weights/best.pt', source='local')
model = torch.hub.load('product_detector', 'custom', path=best_model_path, source='local')
# model['epoch']  # 필요 시 확인 가능

# val 성능 수동 평가 (YOLOv5는 결과 텍스트 저장함)
metrics_path = Path('C:/capstone/yolov5/runs/train/product_detector18/results.csv')
with open(metrics_path) as f:
    lines = f.readlines()
    last_line = lines[-1].strip().split(', ')  # 마지막 epoch 결과

# val 정확도와 손실 값 추출
# 보통 columns: train/val loss, obj_loss, cls_loss, P, R, mAP@0.5, mAP@0.5:0.95
# mAP@0.5 = 7번째, val loss는 앞쪽 평균으로 판단
acc = float(last_line[6])
loss = float(last_line[2])  # 예: obj_loss

# 포맷 맞춰 이름 생성
acc_str = f"{acc:.2f}"
loss_str = f"{loss:.2f}"
new_name = f"pd{acc_str}-{loss_str}.pt"

# 복사해서 저장
target_path = f"C:/capstone/model/{new_name}"
!cp runs/train/product_detector18/weights/best.pt {target_path}
print(f"Saved: {target_path}")

# 모델 성능 평가
!python ./yolov5/val.py --weights C:/capstone/yolov5/runs/train/product_detector2/weights/best.pt --data data.yaml --img 640 --task val --device 0

!python ./product-detector/detect.py --weights ./product-detector/runs/train/product_detector15/weights/best.pt --source C:/capstone/New_sample/aug/images/test --img 640
''']